\
            # 55. 数据分析基础：Pandas（Pandas Fundamentals）

            目标：掌握 Pandas 的核心数据结构与常用数据处理套路：读取数据、清洗、选择、聚合、连接、时间序列与导出。
本章依赖 `pandas`（内部依赖 numpy）；若未安装，会提示安装命令并跳过相关演示。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- NumPy 基础（建议）
- 字典/列表/CSV 概念
- 函数与异常


## 知识点地图

- 1. 为什么需要 Pandas：表格数据处理的主力
- 2. 安装与导入（可选依赖）
- 3. Series 与 DataFrame：索引（index）与列（columns）
- 4. 读取与写出：CSV（可运行，写到 _nb_artifacts）
- 5. 选择与过滤：loc/iloc、布尔条件、列选择
- 6. 缺失值与类型：isna/fillna/dropna/astype
- 7. groupby：分组聚合（统计汇总的核心）
- 8. merge/join/concat：表连接与拼接
- 9. 时间序列入门：to_datetime、dt、resample（了解）
- 10. 性能与可维护性：避免 apply 滥用（提示）


## 自检清单（学完打勾）

- [ ] 理解 Series/DataFrame 的含义与索引（index）概念
- [ ] 会创建 DataFrame/Series 并进行选择（loc/iloc）
- [ ] 会处理缺失值（isna/fillna/dropna）与类型转换
- [ ] 会做 groupby 聚合并理解聚合结果形状
- [ ] 会做 merge/join/concat 进行表连接与拼接
- [ ] 会做时间序列基础：to_datetime、dt、resample（入门）


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：为什么需要 Pandas：表格数据处理的主力

Pandas 提供：
- Series（一维带索引）
- DataFrame（二维表格，列可不同类型）

典型使用场景：
- CSV/Excel/数据库导出数据的清洗与分析
- 特征工程与统计汇总
- 快速验证业务数据逻辑


## 知识点 2：安装与导入（可选依赖）

安装：
- `pip install pandas`

本章代码块都会先尝试 import；若失败会提示，不会中断 notebook。


In [ ]:
try:
    import pandas as pd
except Exception as e:
    pd = None
    print('pandas not available:', type(e).__name__, e)
    print('install: pip install pandas')
else:
    import numpy as np
    print('pandas version:', pd.__version__)
    print('numpy version:', np.__version__)


## 知识点 3：Series 与 DataFrame：索引（index）与列（columns）

- Series：一列数据 + index。
- DataFrame：多列数据 + index + columns。

索引不是“必须连续的 0..n-1”，它可以是日期、字符串等。
索引对选择与对齐运算很重要。


In [ ]:
try:
    import pandas as pd
except Exception:
    print('install pandas to run this cell')
else:
    s = pd.Series([10, 20, 30], index=['a', 'b', 'c'])
    df = pd.DataFrame({'name': ['Alice', 'Bob'], 'age': [18, 20]})
    print('Series:\n', s)
    print('DataFrame:\n', df)


## 知识点 4：读取与写出：CSV（可运行，写到 _nb_artifacts）

常见 I/O：
- `pd.read_csv` / `df.to_csv`
- 也常用 Excel、Parquet（更高效）

这里用 CSV 做最通用的演示，并把文件写入 `_nb_artifacts`。


In [ ]:
from pathlib import Path

try:
    import pandas as pd
except Exception:
    print('install pandas to run this cell')
else:
    ART = Path('_nb_artifacts')
    ART.mkdir(exist_ok=True)

    path = ART / 'people.csv'
    path.write_text('name,age,city\nAlice,18,Beijing\nBob,20,Shanghai\nCarol,,Beijing\n', encoding='utf-8')

    df = pd.read_csv(path)
    print(df)

    out = ART / 'people_out.csv'
    df.to_csv(out, index=False)
    print('wrote', out)


## 知识点 5：选择与过滤：loc/iloc、布尔条件、列选择

- `loc`：基于标签（index/column 名）选择
- `iloc`：基于位置（0..n-1）选择
- 布尔过滤：`df[df['age'] >= 18]`

建议：
- 清晰区分 loc/iloc，避免“看起来能跑但选错行”。


In [ ]:
try:
    import pandas as pd
except Exception:
    print('install pandas to run this cell')
else:
    df = pd.DataFrame({'name':['Alice','Bob','Carol'], 'age':[18,20,17], 'city':['BJ','SH','BJ']})
    print('age>=18:\n', df[df['age'] >= 18])
    print('loc rows 0..1 cols name/age:\n', df.loc[0:1, ['name','age']])
    print('iloc first 2 rows, first 2 cols:\n', df.iloc[:2, :2])


## 知识点 6：缺失值与类型：isna/fillna/dropna/astype

- 缺失值：NaN/NaT
- `isna()` 找缺失
- `fillna` 填充（均值/常量/前向填充）
- `dropna` 丢弃
- `astype` 转类型

注意：
- 缺失值会让整数列变成浮点（因为 NaN 是 float）。可以用 pandas 的可空整数类型（Int64）处理。


In [ ]:
try:
    import pandas as pd
except Exception:
    print('install pandas to run this cell')
else:
    df = pd.DataFrame({'name':['Alice','Bob','Carol'], 'age':[18, None, 20]})
    print('raw dtypes:\n', df.dtypes)
    print(df)
    df['age_filled'] = df['age'].fillna(df['age'].mean())
    print('filled:\n', df)
    df['age_int'] = df['age'].astype('Int64')
    print('nullable int dtypes:\n', df.dtypes)


## 知识点 7：groupby：分组聚合（统计汇总的核心）

- `groupby` + 聚合函数：sum/mean/count/nunique...
- 常见模式：
  - `df.groupby('city')['age'].mean()`
  - `df.groupby(['city','gender']).agg({...})`

注意：聚合结果可能是 Series 或 DataFrame，取决于你选择的列与 agg 方式。


In [ ]:
try:
    import pandas as pd
except Exception:
    print('install pandas to run this cell')
else:
    df = pd.DataFrame({
        'city':['BJ','BJ','SH','SH','SH'],
        'dept':['A','B','A','A','B'],
        'salary':[10, 12, 11, 13, 9],
    })
    print(df.groupby('city')['salary'].mean())
    print(df.groupby(['city','dept']).agg(avg=('salary','mean'), cnt=('salary','size')).reset_index())


## 知识点 8：merge/join/concat：表连接与拼接

- concat：上下/左右拼接
- merge：SQL 风格 join（inner/left/right/outer）

关键：
- 明确主键/关联键
- 关注重复键导致的行数膨胀（笛卡尔放大）


In [ ]:
try:
    import pandas as pd
except Exception:
    print('install pandas to run this cell')
else:
    users = pd.DataFrame({'user_id':[1,2,3], 'name':['A','B','C']})
    orders = pd.DataFrame({'order_id':[10,11,12], 'user_id':[1,1,3], 'amount':[100,200,150]})
    joined = users.merge(orders, on='user_id', how='left')
    print(joined)


## 知识点 9：时间序列入门：to_datetime、dt、resample（了解）

- `pd.to_datetime` 解析时间
- `.dt` 访问器做时间字段提取（year/month/day）
- `resample` 对时间索引按频率聚合（D/W/M）


In [ ]:
try:
    import pandas as pd
except Exception:
    print('install pandas to run this cell')
else:
    df = pd.DataFrame({
        'ts': ['2025-01-01 10:00', '2025-01-01 11:00', '2025-01-02 09:00'],
        'value':[1,2,3]
    })
    df['ts'] = pd.to_datetime(df['ts'])
    df = df.set_index('ts')
    print('by day:\n', df.resample('D').sum())


## 知识点 10：性能与可维护性：避免 apply 滥用（提示）

- 优先用向量化操作（列运算）、内置方法（str/dt/categorical）。
- `apply` 很灵活，但常慢且难并行。
- 大数据量建议：分块读取（chunksize）、更高效格式（Parquet）、或用数据库/分布式计算。


## 常见坑

- 混用 loc/iloc 导致选错数据
- 缺失值导致 dtype 变化（int -> float），下游逻辑出错
- merge 关联键不唯一导致行数爆炸
- groupby 结果形状没看清：Series/DataFrame 搞混
- 滥用 apply：性能差且代码难维护


## 综合小案例：实现一个“销售数据清洗与汇总”小流程

任务：
- 构造一份 sales.csv（包含日期、城市、商品、金额，含缺失值）
- 读取后：清洗缺失、类型转换
- 汇总：按城市/商品 groupby 求总金额与订单数
- 输出结果到 `_nb_artifacts/sales_summary.csv`


In [ ]:
from pathlib import Path

try:
    import pandas as pd
except Exception:
    print('install pandas to run this cell')
else:
    ART = Path('_nb_artifacts')
    ART.mkdir(exist_ok=True)

    sales = ART / 'sales.csv'
    sales.write_text(
        'date,city,sku,amount\n'
        '2025-01-01,BJ,A,10\n'
        '2025-01-01,BJ,B,\n'
        '2025-01-01,SH,A,12\n'
        '2025-01-02,SH,B,9\n'
        '2025-01-02,BJ,A,11\n',
        encoding='utf-8'
    )

    df = pd.read_csv(sales)
    df['date'] = pd.to_datetime(df['date'])
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
    df['amount'] = df['amount'].fillna(0)

    summary = (
        df.groupby(['city','sku'])
          .agg(total_amount=('amount','sum'), orders=('amount','size'))
          .reset_index()
          .sort_values(['city','sku'])
    )

    out = ART / 'sales_summary.csv'
    summary.to_csv(out, index=False)
    print(summary)
    print('wrote', out)


## 自测题（不写代码也能回答）

- Series 与 DataFrame 的差异是什么？
- loc 与 iloc 的差异是什么？
- 缺失值为什么会影响 dtype？如何用可空整数类型解决？
- groupby 的常见输出形状有哪些？你如何快速判断？
- merge 行数暴涨常见原因是什么？如何提前发现？


## 练习题（建议写代码）

- 为 mini_case 增加过滤：只统计金额>0 的订单。
- 给汇总结果增加“占比”列：每个城市内 sku 金额占比。
- 把 sales.csv 换成更大数据：用 chunksize 分块读取并汇总（了解）。
